<a href="https://colab.research.google.com/github/lingzhou0/math412-fall2025/blob/main/homology_of_simplicial_complexes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: Homology of simplicial complexes

Define the homology of simplicial complexes, generalizing the homology of graphs.

This notebook is adjusted from Anibal M. Medina-Mardones's notes: https://colab.research.google.com/drive/1PfId9tnJ3qqfRqriGh2sMYKwiewpEL72?usp=sharing

In [ ]:
import numpy as np
from itertools import combinations

## Directed simplicial complexes

A **directed simplicial complex** is a simplicial complex together with a total order on the vertices of each simplex satisfying the following consistency condition: if $v \lt_{\sigma} w$ in a simplex $\sigma$ then $v \lt_\rho w$ for any face $\rho$ of $\sigma$.

Any simplicial complex can be made into a directed simplicial complex by, for example, choosing a total order of all its vertices.

We will model directed simplicial complexes as ``sets`` of ``tuples`` of vertices. The top simplices will be converted into directed simplicial complexes using the following downward closure method.


In [ ]:
def add_disimplex(dicomplex, disimplex):
    """
    Add a directed simplex to the directed complex along with all its directed faces.

    Parameters:
        simplicial_complex (set): A set of tuples.
        simplex (iterable): An iterable of vertices defining the simplex.
    """
    disimplex = tuple(disimplex)

    # Adds all subtuples of the simplex (excluding the empty tuple)
    for r in range(1, len(disimplex) + 1):
        for diface in combinations(disimplex, r):
            dicomplex.add(tuple(diface))

def dicomplex_from_top_simplices(top_simplices):
    """
    construct the directed simplicial complex from its top directed simplices.

    """
    dicomplex = set()
    for simplex in top_simplices:
        add_disimplex(dicomplex, simplex)
    return dicomplex

### Example

The torus as a directed simplicial complex.
Notice that the vertices are already naturally ordered below.


In [ ]:
torus_top_simplices = [
    [0, 1, 3], [0, 1, 7], [0, 2, 4], [0, 2, 6], [0, 3, 6], [0, 4, 7],
    [1, 2, 5], [1, 2, 8], [1, 3, 5], [1, 7, 8], [2, 5, 6], [2, 4, 8],
    [3, 4, 5], [3, 4, 8], [3, 6, 8], [4, 5, 7], [5, 6, 7], [6, 7, 8]
]

torus = set()
for simplex in torus_top_simplices:
    add_disimplex(torus, simplex)

torus_dicomplex = dicomplex_from_top_simplices(torus_top_simplices)
print(torus_dicomplex)

## Homology

Let us fix a ground field $\Bbbk$ over which all vector spaces will be considered.

Let $X$ be a directed simplicial complex and $X_n$ its set of $n$-dimensional simplices for some non-negative integer $n$.

Define $C_n$ as the vector spaces generated by $X_n$.
Elements in this vector space are referred to as **(degree) $n$-chains**.


Let $\partial_n \colon C_{n} \to C_{n-1}$ be the linear map, referred to as the **boundary map**, defined on basis elements by:

$$
\partial(v_0, \dots, v_n) = \sum_{i=0}^n (-1)^i (v_0, \dots, \widehat{v}_i, \dots, v_n).
$$
And define $\partial_0 = 0$.

Chains in the subvector space $\operatorname{img}(\partial_{n+1})$ are referred to as **(degree) $n$-boundaries**, whereas chains in the subvector space $\operatorname{ker}(\partial_n)$ are referred to as **(degree) $n$-cycles**.

The **(degree) $n$-homology** of $X$ with coefficients in $\Bbbk$, denoted $H_n(X; \Bbbk)$, is the quotient of the $n$-cycles by the $n$-boundaries.
Explicitly,

$$
H_n = \operatorname{ker}(\partial_n) / \operatorname{img}(\partial_{n+1}).
$$

---

#### Fact:

Every boundary is a cycle, i.e., that $\operatorname{img}(\partial_{n+1}) \subseteq \operatorname{ker}(\partial_n)$ or, equivalently, that $\partial_n \circ \partial_{n+1} = 0$.

---

## Boundary matrix

Consider a directed simplicial complex $X$ and a non-negative integer $n$.

Let $A_n$ be the matrix representation of $\partial_n$ with respect to the canonical bases of $X_n$ and $X_{n-1}$ ordered arbitrarily.

Since
$$\operatorname{dim}\mathrm{ker}(\partial_n) = \operatorname{null}(A_n)$$
and
$$\operatorname{dim}\mathrm{img}(\partial_{n+1}) = \operatorname{rank}(A_{n+1})$$
we have
$$
\operatorname{dim} H_n = \operatorname{null}(A_n) - \operatorname{rank}(A_{n+1}).
$$

We refer to $\operatorname{dim} H_n$ as the **(degree) $n$-betti** number of $X$, denoting it $\beta_n$.

The boundary matrices depend on the specific linear orders chosen for the bases.
However, their rank and nullities remains invariant under different orderings, so we generally do not need to be overly concerned with the ordering.

---

#### Fact:

Using the rank-nullity theorem, one can verify that the Euler characteristic of a simplicial complex satisfies:
$$
\chi = \sum_{i \geq 0} (-1)^i\beta_i.
$$

---

In [ ]:
def boundary_matrix(simplicial_complex, n):
    """
    Compute the boundary matrix of degree n for a simplicial complex.

    Parameters:
    - simplicial_complex: set of frozensets, where each frozenset represents a simplex.
    - n: integer, the dimension of simplices for the columns of the matrix.

    Returns:
    - B: numpy array, the boundary matrix of size (num_(n-1)_simplices) x (num_n_simplices).
    - n_simplices: list of n-simplices (columns of B).
    - n_minus_1_simplices: list of (n-1)-simplices (rows of B).
    """
    # Extract simplices of dimension n and (n-1) and canonically order them
    n_simplices = [s for s in simplicial_complex if len(s) == n + 1]
    n_simplices = sorted(n_simplices)
    n_minus_1_simplices = [s for s in simplicial_complex if len(s) == n]
    n_minus_1_simplices = sorted(n_minus_1_simplices)

    # Create an index mapping for (n-1)-simplices
    n_minus_1_simplex_index = {s: i for i, s in enumerate(n_minus_1_simplices)}

    # Initialize the boundary matrix
    A = np.zeros((len(n_minus_1_simplices), len(n_simplices)), dtype=int)

    # Fill the boundary matrix
    if n != 0:
        for j, simplex in enumerate(n_simplices):
            for pos, face in enumerate(combinations(simplex, n)):
                sign = (-1)**(pos + n)  # Use consistent behaviour of combinations
                A[n_minus_1_simplex_index[face], j] = sign

    return A, n_simplices, n_minus_1_simplices


Let us inspect the boundary matrices of the torus.

In [ ]:
n = 2
A, n_simplices, n_minus_1_simplices = boundary_matrix(torus, n)
print(f"{n}-boundary matrix:\n")
print(A)

Let us check that the every boundary is a cycle.
This is equivalent to show that $\partial_n \circ \partial_{n+1} = 0$, or, in terms of matrices, that $A_nA_{n+1} = 0$.

In [ ]:
n = 1
A1, _, _ = boundary_matrix(torus, n)
A2, _, _ = boundary_matrix(torus, n+1)
print(np.dot(A1, A2))

## Homology and betti numbers

Let us now compute the homology of some complexes.
We will work over the field with two elements.
We first need to implement a column reduction function for binary matrices.
Let us first define the top simplices of the directed simplicial complexes we will work with.

In [ ]:
names = ["cicle", "sphere", "torus", "Klein", "rp2"]

circle_top_simplices = [
    [0,1], [0,2], [1,2]
]

sphere_top_simplices = [
    [0,1,2], [0,1,3], [0,2,3], [1,2,3]
]

torus_top_simplices = [
    [0, 1, 3], [0, 1, 7], [0, 2, 4], [0, 2, 6], [0, 3, 6], [0, 4, 7],
    [1, 2, 5], [1, 2, 8], [1, 3, 5], [1, 7, 8], [2, 5, 6], [2, 4, 8],
    [3, 4, 5], [3, 4, 8], [3, 6, 8], [4, 5, 7], [5, 6, 7], [6, 7, 8]
]

klein_top_simplices = [
    [2,3,7], [1,2,3], [1,3,5], [1,5,7],
    [1,4,7], [2,4,6], [1,2,6], [1,6,0],
    [1,4,0], [2,4,0], [3,4,7], [3,4,6],
    [3,5,6], [5,6,0], [2,5,0], [2,5,7]
]

rp2_top_simplices = [
    [0,1,2], [0,2,3], [0,1,5], [0,4,5],
    [0,3,4], [1,2,4], [1,3,4], [1,3,5],
    [2,3,5], [2,4,5]
]

# collect all in one list
all_top_simplices = [
    circle_top_simplices,
    sphere_top_simplices,
    torus_top_simplices,
    klein_top_simplices,
    rp2_top_simplices
]

Let us recall the reduction implementation over mod 2 coefficients.

In [ ]:
def _pivot(column):
    """
    Find the index of the first nonzero entry in a column.

    Args:
        column (numpy.ndarray): A 1D NumPy array representing a column of the matrix.

    Returns:
        int: The index of the first nonzero entry.
             Returns -1 if the column consists entirely of zeros.
    """
    nonzero_indices = np.where(column)[0]
    return nonzero_indices[0] if nonzero_indices.size > 0 else -1

def reduce_matrix(matrix):
    """
    Perform column reduction on a binary matrix over the field ℤ/2ℤ (mod 2).

    The reduction follows a variant of Gaussian elimination tailored for binary matrices.
    Columns with the same leading (pivot) entry are XOR-ed to eliminate redundancy.

    Args:
        matrix (numpy.ndarray): A 2D boolean NumPy array representing the matrix to be reduced.

    Returns:
        numpy.ndarray: A reduced binary matrix, where each column has a unique pivot (if possible).
    """
    col_num = matrix.shape[1]  # Number of columns in the matrix
    reduced = np.array(matrix, dtype=bool)  # Ensure the matrix is binary (mod 2)
    i = -1

    while i < col_num - 1:  # Iterate over columns
        i += 1
        if _pivot(reduced[:, i]) == -1:  # Skip zero columns
            continue

        j = i
        while j < col_num - 1:  # Iterate over all later columns
            j += 1
            piv_i = _pivot(reduced[:, i])
            piv_j = _pivot(reduced[:, j])

            if piv_i == piv_j:  # If two columns share the same pivot, reduce the later one
                reduced[:, j] = np.logical_xor(reduced[:, j], reduced[:, i])
                i = -1  # Restart outer while loop to check all columns again

    return reduced

To compute the betti numbers we read off the rank and nullity of the reduced matrices.

In [ ]:
def betti_number(complex, n):
    """
    Compute the Betti number of a simplicial complex at a given dimension
    using mod 2 coefficients

    Parameters:
    - complex: set of frozensets, where each frozenset represents a simplex.
    - n: integer, the degree of the computed homology.

    Returns:
    - betti: integer, the degree n Betti number.
    """
    B1, _, _ = boundary_matrix(complex, n)
    reduced_B1 = reduce_matrix(B1)
    nullity = np.sum(np.all(reduced_B1 == 0, axis=0))

    B2, _, _ = boundary_matrix(complex, n+1)
    reduced_B2 = reduce_matrix(B2)
    rank = np.sum(np.any(reduced_B2, axis=0))

    return nullity - rank

Let us now compute the Betti numbers of our complexes in degrees 0, 1, 2.

In [ ]:
print("Betti  | 0 | 1 | 2")
print("-"*18)

# The list all_top_simplices stores the set of top-dimensional simplices for each space
for name, top_simplices in zip(names, all_top_simplices):

    # This line computes the (directed) simplicial complex from the its top-dimensional simplices
    dicomplex = dicomplex_from_top_simplices(top_simplices)

    # The following lines compute and print the betti numbers
    b0 = betti_number(dicomplex, 0)
    b1 = betti_number(dicomplex, 1)
    b2 = betti_number(dicomplex, 2)
    print(f"{name:<6} | {b0} | {b1} | {b2}")

# Section 2: Homework questions

#### **Warm-up (review)**:
Run the examples already in the notebook (circle, sphere, torus, Klein bottle and real projective space) and record the Betti numbers of the examples computed.

#### **Two loops at a point**:

Construct a simplicial complex that represents two circles joined at a single point. Compute its Betti numbers using the provided code.

#### **Torus with one vertex removed**:

Modify the simplicial complex for the torus by removing vertex 0 and all simplices containing it. Recompute the Betti numbers of this new complex.

#### **Compare**:
Compare the Betti numbers from part (b) and part (c). What do you observe? Briefly explain why this relation holds.

#### **Another example**:
Compute the Betti numbers of the simplicial complex given in HW\#1 Problem 8.

#### **Bonus**:
Modify the `reduce_matrix` function to be over real numbers instead of $\mathbb{Z}_2$. Use the modified code to recompute the Betti numbers for torus, Klein bottle and real projective space.